<a href="https://colab.research.google.com/github/satyasri77/chat_bot/blob/main/new_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-cpp-python sentence-transformers faiss-cpu huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 MB 27.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
pip install -U langchain-community

In [ ]:
from huggingface_hub import hf_hub_download

HF_TOKEN = "your_token"  # ← Replace with your Hugging Face token

model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    token=HF_TOKEN
)


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("customer_service_info.txt")  # ← Upload this file first
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 1})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_cpp import Llama

llm = Llama(model_path=model_path, n_ctx=1024, verbose=False)


llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


In [ ]:
def ask(question):
    # Step 1: Retrieve top relevant chunks
    context_docs = retriever.get_relevant_documents(question)
    context = "\n".join(doc.page_content for doc in context_docs)

    # Step 2: If nothing relevant was found, return fallback message
    if not context.strip():
        return "I'm only able to answer customer support related questions."

    # Step 3: Build a strict prompt using only the retrieved context
    prompt = f"""
You are a helpful customer support assistant.
Only answer based on the context below. Do not use outside knowledge.
If the answer is not related to customer support queries, reply with:
"I'm only able to answer customer support related questions."

Context:
{context}

Question: {question}
Answer:"""

    response = llm(prompt, max_tokens=200,temperature=0,top_p=0.8, top_k=40)
    return response["choices"][0]["text"].strip()


In [ ]:
print("💬 LLaMA RAG Chatbot Ready! Type 'exit' to stop.\n")
while True:
    q = input("You: ")
    if q.lower() in ['exit', 'quit']:
        break
    print("Bot:", ask(q), "\n")

💬 LLaMA RAG Chatbot Ready! Type 'exit' to stop.

You: write a program for palindrome


/tmp/ipython-input-10-3799027340.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = retriever.get_relevant_documents(question)


Bot: I'm only able to answer customer support related questions. 

You: what are new updates in news?
Bot: I'm sorry, but I am not able to provide information about news updates. I am only able to answer customer support related questions. 

You: what is return policy?
Bot: Our return policy allows customers to return items within 30 days of receipt. The items must be unused and in their original packaging, and proof of purchase is required. Refunds are processed within 5–7 business days after receiving the returned item. Certain items may be non-refundable. For return instructions, visit our returns portal at returns.yourcompany.com. 

You: exit
